In [1]:
using RigidBodyDynamics
using RigidBodySim
using MeshCatMechanisms
using StrandbeestRobot

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/998a51f82a651317b2614fbfe83ea114061d742e-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/998a51f82a651317b2614fbfe83ea114061d742e-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [4]:
mechanism = StrandbeestRobot.mechanism()
gui = GUI(mechanism, URDFVisuals(StrandbeestRobot.urdfpath()))

┌ Warning: Initial position cannot be on the boundary of the box. Moving elements to the interior.
│ Element indices affected: [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49, 51, 52, 54, 55, 57, 58, 60, 61, 63, 64, 66, 67, 69, 70, 72, 73, 75, 76, 78, 79]
└ @ Optim /home/rdeits/.julia/packages/Optim/fabGe/src/multivariate/solvers/constrained/fminbox.jl:212


In [ ]:
#NBSKIP
open(gui)

In [ ]:
state = MechanismState(mechanism)
StrandbeestRobot.solve_initial_configuration!(state)

floatingjoint = findjoint(mechanism, "floating_base")
set_configuration!(state, floatingjoint, [1., 0, 0, 0, 0, 0, 0.9])

copyto!(gui.visualizer, state)

In [6]:
# Simulate the robot
control! = let I = velocity_range(state, findjoint(mechanism, "joint_crossbar_crank"))
    function (τ, t, state)
        τ[I] .= -10.0 .* (velocity(state)[I] .- 2.0)
    end
end
dynamics = Dynamics(mechanism, control!)
problem = ODEProblem(dynamics, state, (0, 5.0))
@time solution = solve(problem, Tsit5(), abs_tol=1e-6, dt=1e-6)
setanimation!(gui.visualizer, solution)

 30.809956 seconds (4.06 M allocations: 802.651 MiB, 0.65% gc time)
